In [1]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas as pd
import lightgbm as lgb
import numpy as np

In [2]:
active_losses_df = pd.read_csv(
    "../data/Avtice-losses.csv", skiprows=2, names=["datetime", "MWh"], parse_dates=["datetime"], index_col="datetime")

active_losses_df.index = active_losses_df.tz_localize('Europe/Brussels', ambiguous="infer").tz_convert('UTC').index
active_losses_df.index = active_losses_df.index - pd.Timedelta(minutes=15)
active_losses_df = active_losses_df.resample('1H').sum()
active_losses_df["MWh"] = active_losses_df["MWh"]/1000
active_losses_df

,MWh
datetime,
2018-12-31 23:00:00+00:00,139.525004
2019-01-01 00:00:00+00:00,129.716036
2019-01-01 01:00:00+00:00,133.398074
2019-01-01 02:00:00+00:00,135.133852
2019-01-01 03:00:00+00:00,131.699424
...,...
2021-12-31 18:00:00+00:00,171.707318
2021-12-31 19:00:00+00:00,159.462903
2021-12-31 20:00:00+00:00,155.109520


# Predict next hour based on last 7 days

In [3]:
df1 = active_losses_df.copy()
feature_columns = ['MWh']

In [4]:
# Create lagged features
for col in feature_columns:
    for i in range(1, 7 * 24 + 1):  # for past 7 days (in hours)
        df1[f'lag_{col}_{i}'] = df1[col].shift(i)

/tmp/ipykernel_99872/4144301521.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[f'lag_{col}_{i}'] = df1[col].shift(i)
/tmp/ipykernel_99872/4144301521.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1[f'lag_{col}_{i}'] = df1[col].shift(i)
/tmp/ipykernel_99872/4144301521.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented fram

In [5]:
df1

,MWh,lag_MWh_1,lag_MWh_2,lag_MWh_3,lag_MWh_4,lag_MWh_5,lag_MWh_6,lag_MWh_7,lag_MWh_8,lag_MWh_9,...,lag_MWh_159,lag_MWh_160,lag_MWh_161,lag_MWh_162,lag_MWh_163,lag_MWh_164,lag_MWh_165,lag_MWh_166,lag_MWh_167,lag_MWh_168
datetime,,,,,,,,,,,,,,,,,,,,,
2018-12-31 23:00:00+00:00,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 00:00:00+00:00,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 01:00:00+00:00,133.398074,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 02:00:00+00:00,135.133852,133.398074,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 03:00:00+00:00,131.699424,135.133852,133.398074,129.716036,139.525004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-31 18:00:00+00:00,171.707318,152.159763,162.995636,188.541866,176.617953,167.343565,145.792085,152.648212,150.967027,149.349078,...,125.661865,119.124221,115.795738,103.814345,97.836516,97.016161,103.763064,105.380492,103.315829,99.421761
2021-12-31 19:00:00+00:00,159.462903,171.707318,152.159763,162.995636,188.541866,176.617953,167.343565,145.792085,152.648212,150.967027,...,136.132174,125.661865,119.124221,115.795738,103.814345,97.836516,97.016161,103.763064,105.380492,103.315829
2021-12-31 20:00:00+00:00,155.109520,159.462903,171.707318,152.159763,162.995636,188.541866,176.617953,167.343565,145.792085,152.648212,...,144.984402,136.132174,125.661865,119.124221,115.795738,103.814345,97.836516,97.016161,103.763064,105.380492


In [6]:
X1 = df1.drop('MWh', axis=1)  # Features
y1 = df1['MWh']  # Target variable

X_train1 = X1[168:21000]
y_train1 = y1[168:21000]
X_test1 = X1[21168:]
y_test1 = y1[21168:]

In [ ]:
# Define the parameters for LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'device': 'gpu'  # Enable GPU support
}

# Create the LightGBM dataset
train_data1 = lgb.Dataset(X_train1, label=y_train1)
# Train the model
model1 = lgb.train(params, train_data1)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 42840
[LightGBM] [Info] Number of data points in the train set: 20832, number of used features: 168
[LightGBM] [Info] Using GPU Device: NVIDIA A100 80GB PCIe, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 168 dense feature groups (3.34 MB) transferred to GPU in 0.033963 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 107.167990


In [ ]:
y_pred1 = model1.predict(X_test1)
metrics.mean_absolute_error(y_test1, y_pred1)

# Predict value in 24 hours based on previous 7 days

In [ ]:
df24 = active_losses_df.copy()
feature_columns = ['MWh']

In [ ]:
# Create lagged features
for col in feature_columns:
    for i in range(24, 8 * 24 + 1):  # for past 7 days (in hours)
        df24[f'lag_{col}_{i}'] = df24[col].shift(i)

In [ ]:
df24

In [ ]:
X24 = df24.drop('MWh', axis=1)  # Features
y24 = df24['MWh']  # Target variable

X_train24 = X24[194:21000]
y_train24 = y24[194:21000]
X_test24 = X24[21194:]
y_test24 = y24[21194:]

In [ ]:
# Define the parameters for LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'device': 'gpu'  # Enable GPU support
}

# Create the LightGBM dataset
train_data24 = lgb.Dataset(X_train24, label=y_train24)
# Train the model
model24 = lgb.train(params, train_data24)

In [ ]:
y_pred24 = model24.predict(X_test24)
metrics.mean_absolute_error(y_test24, y_pred24)

# Predict next 24 hours using predictions

In [8]:
def perdict_next_24hours(data_df: pd.DataFrame, date_input, model):
    """
    Predict next hour based on last 7 days
    Uses preditcion to computes next 24 hours
    """
    input_data = data_df.loc[data_df.index<date_input].tail(1)
    y_pred = []
    
    for i in range(0,24):
        y_pred += model.predict(input_data)
        
        #Remove first column
        input_data = input_data[1:]
        input_data = input_data.shift(periods=1, axis="columns")
        input_data.iloc[:,0] = y_pred
        print(input_data)
        
    return y_pred

In [9]:
df = active_losses_df.copy()
feature_columns = ['MWh']
for col in feature_columns:
    for i in range(1, 7 * 24 + 1):  # for past 7 days (in hours)
        df[f'lag_{col}_{i}'] = df[col].shift(i)

/tmp/ipykernel_82785/2424762308.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_82785/2424762308.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'lag_{col}_{i}'] = df[col].shift(i)
/tmp/ipykernel_82785/2424762308.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, u

In [10]:
X = df.drop('MWh', axis=1)  # Features
y = df['MWh']  # Target variable

X_train = X[168:21000]
y_train = y[168:21000]
X_test = X[21168:]
y_test = y[21168:]

In [12]:
# Define the parameters for LightGBM
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'device': 'gpu'  # Enable GPU support
}

# Create the LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
# Train the model
model = lgb.train(params, train_data)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.671838 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 42840
[LightGBM] [Info] Number of data points in the train set: 20832, number of used features: 168
[LightGBM] [Info] Start training from score 107.167990


KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
metrics.mean_absolute_error(y_test, y_pred)

In [ ]:
perdict_next_24hours(data_df=df, date_input=X_test.index[1], model=model)

# Using Vectors

In [ ]:
# Create a new DataFrame with a date index
date_index = active_losses_df.index.date
new_df = pd.DataFrame(index=date_index)

# Initialize an empty array for each date with 24 values
new_df['MWh'] = [np.array([np.nan] * 24) for _ in range(len(date_index))]

# Populate the array with the hourly values
for date in date_index:
    #hourly_values = active_losses_df.loc[date.strftime('%Y-%m-%d')]['MWh'].values
    #new_df.at[date, 'MWh'][:len(hourly_values)] = hourly_values
    
    date_values = active_losses_df.loc[date.strftime('%Y-%m-%d')]
    #print(date_values)
    #print(date_values['MWh'].values)
    new_df.loc[date] = date_values.values

# Optional: Fill NaN values with zeros
new_df['MWh'] = new_df['MWh'].apply(lambda x: np.nan_to_num(x, nan=0))

# Print the resulting DataFrame
#print(new_df) 

In [9]:
df.reset_index()

,datetime,MWh
0,2018-12-31 23:00:00+00:00,39.143346
1,2018-12-31 23:15:00+00:00,32.788069
2,2018-12-31 23:30:00+00:00,33.018916
3,2018-12-31 23:45:00+00:00,34.574673
4,2019-01-01 00:00:00+00:00,33.417096
...,...,...
105211,2021-12-31 21:45:00+00:00,40.720617
105212,2021-12-31 22:00:00+00:00,38.156039
105213,2021-12-31 22:15:00+00:00,36.290837
105214,2021-12-31 22:30:00+00:00,35.947704


In [10]:
# Adding a second index that repeats from 1 to 8
df['sample_index'] = (df.reset_index().index // 8) + 1

In [12]:
df

,MWh,sample_index
datetime,,
2018-12-31 23:00:00+00:00,39.143346,1
2018-12-31 23:15:00+00:00,32.788069,1
2018-12-31 23:30:00+00:00,33.018916,1
2018-12-31 23:45:00+00:00,34.574673,1
2019-01-01 00:00:00+00:00,33.417096,1
2019-01-01 00:15:00+00:00,34.716826,1
2019-01-01 00:30:00+00:00,31.063598,1
2019-01-01 00:45:00+00:00,30.518516,1
2019-01-01 01:00:00+00:00,33.344467,2
